In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

try:
    df = pd.read_csv("Bitcoin Historical Data2.csv", sep=",", encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv("Bitcoin Historical Data2.csv", sep=",", encoding='latin1')

for column in df.columns:
    df[column] = df[column].str.replace(',', '')
for column in df.columns:
    df[column] = df[column].str.replace('%', '')
for column in df.columns:
    df[column] = df[column].str.replace('/', '-')
df['Vol.'] = df['Vol.'].str.replace('K', 'e3')
df['Vol.'] = df['Vol.'].str.replace('M', 'e6')
df['Vol.'] = df['Vol.'].str.replace('B', 'e9')
for column in df.columns:
  if column != 'Date':
    df[column] = df[column].astype(float)

In [ ]:
print(df.head(1))

         Date    Price     Open     High      Low     Vol.  Change %
0  03-02-2024  61994.5  62397.7  62446.3  61621.9  33800.0     -0.65


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold, ShuffleSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from numpy import sqrt
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint

df = df.dropna()

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

X = df.drop(['Price'], axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<ipython-input-65-7ddc3ef4bc17>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])


In [ ]:

k = 5
knn = KNeighborsRegressor(n_neighbors=k)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error для K={k}: ", mse)

Mean Squared Error для K=5:  182785.01186572865


In [ ]:
param_grid = {'n_neighbors': np.arange(1, 31)}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)
print("Лучший параметр K с GridSearchCV: ", grid_search.best_params_)

random_search = RandomizedSearchCV(KNeighborsRegressor(), param_distributions=param_grid, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train_scaled, y_train)
print("Лучший параметр K с RandomizedSearchCV: ", random_search.best_params_)

Лучший параметр K с GridSearchCV:  {'n_neighbors': 2}
Лучший параметр K с RandomizedSearchCV:  {'n_neighbors': 1}


In [ ]:
best_k = grid_search.best_estimator_
y_pred_optimal = best_k.predict(X_test_scaled)

mse_optimal = mean_squared_error(y_test, y_pred_optimal)
print(f"Mean Squared Error для оптимального K: ", mse_optimal)

Mean Squared Error для оптимального K:  156540.58844723616


In [ ]:
print(f"Исходный MSE: {mse}, оптимальный MSE: {mse_optimal}")

Исходный MSE: 182785.01186572865, оптимальный MSE: 156540.58844723616
